## Import modules

In [1]:
#importing the libraries and files
import pandas as pd
import numpy as np
import Recommenders as Recommenders

## Loading the dataset

In [2]:
#Reading the triplets_file file
# song_df_1 = pd.read_csv('triplets_file.csv')
song_df_1.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [3]:
#Reading the song_data file
# song_df_2 = pd.read_csv('song_data.csv')
song_df_2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [4]:
# combining both the data files 
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on='song_id', how='left')
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [5]:
#calculating the lengths of both the heads.
print(len(song_df_1), len(song_df_2))

2000000 1000000


In [6]:
len(song_df)

2000000

## Data Preprocessing

In [7]:
# creating new feature combining title and artist name
song_df['song'] = song_df['title']+' - '+song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly - Foo Fighters


In [8]:
# taking top 10k samples for quick results
song_df = song_df.head(10000)

In [9]:
# cummulative sum of listen count of the songs
song_grouped = song_df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#40 - DAVE MATTHEWS BAND,1
1,& Down - Boys Noize,4
2,'97 Bonnie & Clyde - Eminem,2
3,'Round Midnight - Miles Davis,3
4,'Till I Collapse - Eminem / Nate Dogg,6


In [10]:
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
3660,Sehr kosmisch - Harmonia,45,0.45
4678,Undo - Björk,32,0.32
5105,You're The One - Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.28
3655,Secrets - OneRepublic,28,0.28
...,...,...,...
5139,high fives - Four Tet,1,0.01
5140,in white rooms - Booka Shade,1,0.01
5143,paranoid android - Christopher O'Riley,1,0.01
5149,¿Lo Ves? [Piano Y Voz] - Alejandro Sanz,1,0.01


## Popularity Recommendation Engine

In [11]:
#creating an 0bject pr and calling the function popularity_recommender from recommender class
pr = Recommenders.popularity_recommender_py()

In [12]:
pr.create(song_df, 'user_id', 'song')

In [13]:
# displaying the top 10 popular songs from the subset data
pr.recommend(song_df['user_id'][50])

,user_id,song,score,Rank
3660,bd4c6e843f00bd476847fb75c47b4fb430a06856,Sehr kosmisch - Harmonia,45,1.0
4678,bd4c6e843f00bd476847fb75c47b4fb430a06856,Undo - Björk,32,2.0
5105,bd4c6e843f00bd476847fb75c47b4fb430a06856,You're The One - Dwight Yoakam,32,3.0
1071,bd4c6e843f00bd476847fb75c47b4fb430a06856,Dog Days Are Over (Radio Edit) - Florence + Th...,28,4.0
3655,bd4c6e843f00bd476847fb75c47b4fb430a06856,Secrets - OneRepublic,28,5.0
4378,bd4c6e843f00bd476847fb75c47b4fb430a06856,The Scientist - Coldplay,27,6.0
4712,bd4c6e843f00bd476847fb75c47b4fb430a06856,Use Somebody - Kings Of Leon,27,7.0
3476,bd4c6e843f00bd476847fb75c47b4fb430a06856,Revelry - Kings Of Leon,26,8.0
1387,bd4c6e843f00bd476847fb75c47b4fb430a06856,Fireflies - Charttraxx Karaoke,24,9.0
1862,bd4c6e843f00bd476847fb75c47b4fb430a06856,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,10.0


In [15]:
#Since these are top 10 most popular songs, so we'll get the same result for every user irrespective of the ID.
pr.recommend(song_df['user_id'][100])

,user_id,song,score,Rank
3660,e006b1a48f466bf59feefed32bec6494495a4436,Sehr kosmisch - Harmonia,45,1.0
4678,e006b1a48f466bf59feefed32bec6494495a4436,Undo - Björk,32,2.0
5105,e006b1a48f466bf59feefed32bec6494495a4436,You're The One - Dwight Yoakam,32,3.0
1071,e006b1a48f466bf59feefed32bec6494495a4436,Dog Days Are Over (Radio Edit) - Florence + Th...,28,4.0
3655,e006b1a48f466bf59feefed32bec6494495a4436,Secrets - OneRepublic,28,5.0
4378,e006b1a48f466bf59feefed32bec6494495a4436,The Scientist - Coldplay,27,6.0
4712,e006b1a48f466bf59feefed32bec6494495a4436,Use Somebody - Kings Of Leon,27,7.0
3476,e006b1a48f466bf59feefed32bec6494495a4436,Revelry - Kings Of Leon,26,8.0
1387,e006b1a48f466bf59feefed32bec6494495a4436,Fireflies - Charttraxx Karaoke,24,9.0
1862,e006b1a48f466bf59feefed32bec6494495a4436,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,10.0


## Item Similarity Recommendation

In [14]:
ir = Recommenders.item_similarity_recommender_py()
ir.create(song_df, 'user_id', 'song')

In [19]:
user_items = ir.get_user_items(song_df['user_id'][5000])

In [20]:
# display user songs history
for user_item in user_items:
    print(user_item)

A Pain That Im Used To - Depeche Mode
Natural's Not In It - Gang Of Four
Pogo - Digitalism
Uprising - Muse
That Was Just A Dream - Cut Copy
Five Minutes To Midnight - Boys Like Girls
Losing Touch - The Killers
The Ballad of Michael Valentine - The Killers
Epona - Enya
Nine Times Out Of Ten (1998 Digital Remaster) - Cliff Richard & The Shadows
The Killing Moon - Echo And The Bunnymen
Enterlude - The Killers
Under The Gun - The Killers
Hey - Pixies
It's No Good (Single Version) - Depeche Mode
Black Is My Color - I-Roy
O Samba E O Tango - Caetano Veloso
Lights & Music - Cut Copy
Saturdays - Cut Copy
This Charming Man - The Smiths
I Need A Dollar - Aloe Blacc
The Power of Love - Frankie Goes To Hollywood
Ruby_ Don't Take Your Love To Town - The Killers
Cemetery Gates - The Smiths
The Way You Do The Things You Do (2009 Digital Remaster) - UB40
Suena tu guitarra - Fernando Soto
Hearts On Fire - Cut Copy
Feel The Love - Cut Copy
Notorious - Duran Duran
Add It Up - Violent Femmes
The Meaning 

In [21]:
# give song recommendation for that user
ir.recommend(song_df['user_id'][5000])

No. of unique songs for the user: 133
no. of unique songs in the training set: 5151
Non zero values in cooccurence_matrix :36583


,user_id,song,score,rank
0,e21477efb83bd323205ce6f5bd662f3df9d477e5,So Haunted - Cut Copy,0.064411,1
1,e21477efb83bd323205ce6f5bd662f3df9d477e5,Watch The Tapes - LCD Soundsystem,0.045632,2
2,e21477efb83bd323205ce6f5bd662f3df9d477e5,Work On You - MSTRKRFT,0.045632,3
3,e21477efb83bd323205ce6f5bd662f3df9d477e5,Shine Shine - Boys Noize,0.045632,4
4,e21477efb83bd323205ce6f5bd662f3df9d477e5,Losing My Edge - LCD Soundsystem,0.045632,5
5,e21477efb83bd323205ce6f5bd662f3df9d477e5,The Pulse - Digitalism,0.045632,6
6,e21477efb83bd323205ce6f5bd662f3df9d477e5,The Battery - Boys Noize,0.045632,7
7,e21477efb83bd323205ce6f5bd662f3df9d477e5,Car Wash - Rose Royce,0.045023,8
8,e21477efb83bd323205ce6f5bd662f3df9d477e5,Silent Shout - The Knife,0.045023,9
9,e21477efb83bd323205ce6f5bd662f3df9d477e5,Us V Them - LCD Soundsystem,0.045023,10


In [22]:
# give related songs based on the words
ir.get_similar_items(['Shine Shine - Boys Noize'])

no. of unique songs in the training set: 5151
Non zero values in cooccurence_matrix :570


,user_id,song,score,rank
0,,Watch The Tapes - LCD Soundsystem,1.00,1
1,,Work On You - MSTRKRFT,1.00,2
2,,Losing My Edge - LCD Soundsystem,1.00,3
3,,The Pulse - Digitalism,1.00,4
4,,The Battery - Boys Noize,1.00,5
5,,Disco Infiltrator (FK's Infiltrated Vocal) - L...,0.75,6
6,,& Down - Boys Noize,0.75,7
7,,Oh! - Boys Noize,0.75,8
8,,Too Much Love - LCD Soundsystem,0.75,9
9,,Easy Love - MSTRKRFT,0.75,10
